# Experiment with parameters for a Ridge Regression Model on the Diabetes Dataset

This notebook is for experimenting with different parameters to train a ridge regression model on the Diabetes dataset.

In [1]:
# Change out of the experimentation directory
%cd ..

C:\Users\brysmith\Source\Repos\MLOpsPython


In [2]:
import azureml.core
from azureml.core import Workspace

In [3]:
# Load the workspace from the saved config file
ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...
WARNING - Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/42cc3295-cd0e-449c-b98e-5ce5b560c1d3', 'tenant_id': '42cc3295-cd0e-449c-b98e-5ce5b560c1d3'}' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS50158: External security challenge not satisfied.\r\nTrace ID: 1e188174-b55f-449a-b42f-b60752d63300\r\nCorrelation ID: 976b40a0-2d3f-4848-8220-dc9dafabac4e\r\nTimestamp: 2020-03-03 20:09:21Z","error_codes":[50158],"timestamp":"2020-03-03 20:09:21Z","trace_id":"1e188174-b55f-449a-b42f-b60752d63300","correlation_id":"976b40a0-2d3f-4848-8220-dc9dafabac4e","error_uri":"https://login.microsoftonline.com/error?code=50158","suberror":"basic_action"}'


Interactive authentication successfully completed.


In [4]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'diabetes-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

# Copy the train functions into the experiment folder
shutil.copy('diabetes_regression/training/train.py', os.path.join(training_folder, "train.py"))

'diabetes-training\\train.py'

In [5]:
%%writefile $training_folder/parameters.json
{
    "training":
    {
        "alpha": 0.3
    },
    "evaluation":
    {

    },
    "scoring":
    {
        
    }
}


Writing diabetes-training/parameters.json


In [6]:
%%writefile $training_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import shutil

from train import split_data, train_model

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
train_df = pd.read_csv('diabetes.csv')

data = split_data(train_df)

# Specify the parameters to test
with open("parameters.json") as f:
    pars = json.load(f)
    train_args = pars["training"]

# Log parameters
for k, v in train_args.items():
    run.log(k, v)

model, metrics = train_model(data, train_args)

# Log metrics
for k, v in metrics.items():
    run.log(k, v)

# Save the parameters file to the outputs folder
os.makedirs('outputs', exist_ok=True)
shutil.copy('parameters.json', os.path.join('outputs', 'parameters.json'))
    
run.complete()

Writing diabetes-training/diabetes_training.py


In [7]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# Create an estimator
estimator = Estimator(source_directory=training_folder,
                      entry_script='diabetes_training.py',
                      compute_target='local',
                      conda_packages=['scikit-learn']
                      )

# Create an experiment
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

RunId: diabetes-training_1583266166_a12fa3dc
Web View: https://mlworkspace.azure.ai/portal/subscriptions/48d404f6-3a69-4552-a210-b1afe5537cc1/resourceGroups/mlopsohrg/providers/Microsoft.MachineLearningServices/workspaces/mlopsoh-ws/experiments/diabetes-training/runs/diabetes-training_1583266166_a12fa3dc

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 14668
Running: ['cmd.exe', '/c', 'C:\\Users\\brysmith\\AppData\\Local\\Temp\\azureml_runs\\diabetes-training_1583266166_a12fa3dc\\azureml-environment-setup/docker_env_checker.bat']

Materialized image not found on target: azureml/azureml_942e102c8fd48681f49452a15f3fb0f4


Logging experiment preparation status in history service.
Running: ['cmd.exe', '/c', 'C:\\Users\\brysmith\\AppData\\Local\\Temp\\azureml_runs\\diabetes-training_1583266166_a12fa3dc\\azureml-environment-setup/docker_env_builder.bat']
Running: ['

  Created wheel for json-logging-py: filename=json_logging_py-0.2-py3-none-any.whl size=3923 sha256=d2389923d5fa749959e34ba99e193cfca7c52e3d6157eea15c3fc81f6fe26a52
  Stored in directory: /root/.cache/pip/wheels/e2/1d/52/535a274b9c2ce7d4064838f2bdb62013801281ef7d7f21e2ee
  Created wheel for liac-arff: filename=liac_arff-2.4.0-py3-none-any.whl size=13333 sha256=e0cc6e1444ba030cc8494be10458deddbad063265c56cbe0a61b7cfe1f4391b1
  Stored in directory: /root/.cache/pip/wheels/ba/2a/e1/6f7be2e2ea150e2486bff64fd6f0670f4f35f4c8f31c819fb8
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78530 sha256=71bb549eeb11790f4c5b80740982bfaf3f97e22e38d7b630935a76ce02d07da1
  Stored in directory: /root/.cache/pip/wheels/09/84/74/d2b4feb9ac9488bc83c475cb2cbe8e8b7d9cea8320d32f3787
  Created wheel for pycparser: filename=pycparser-2.19-py2.py3-none-any.whl size=111031 sha256=e21fb51230c57b5ba0a2ee68ce1b31d347bae178a3d717f7a061e39c78c5754a
  Stored in directory: /root/.cache/pip/wheels/c6/


Streaming azureml-logs/70_driver_log.txt

Starting the daemon thread to refresh tokens in background for process with pid = 8
Entering Run History Context Manager.
Preparing to call script [ diabetes_training.py ] with arguments: []
After variable expansion, calling script [ diabetes_training.py ] with arguments: []

Loading Data...


The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 8
Cleaning up all outstanding Run operations, waiting 300.0 seconds
2 items cleaning up...
Cleanup took 0.01858043670654297 seconds

Execution Summary
RunId: diabetes-training_1583266166_a12fa3dc
Web View: https://mlworkspace.azure.ai/portal/subscriptions/48d404f6-3a69-4552-a210-b1afe5537cc1/resourceGroups/mlopsohrg/providers/Microsoft.MachineLearningServices/workspaces/mlopsoh-ws/experiments/diabetes-training/runs/diabetes-training_1583266166_a12fa3dc



{'runId': 'diabetes-training_1583266166_a12fa3dc',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-03-03T20:12:24.785499Z',
 'endTimeUtc': '2020-03-03T20:12:33.184738Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '506b34b4-2bdd-42b8-b717-9c0618480bfb',
  'azureml.git.repository_uri': 'https://github.com/microsoft/MLOpsPython.git',
  'mlflow.source.git.repoURL': 'https://github.com/microsoft/MLOpsPython.git',
  'azureml.git.branch': 'jotaylo/split_train_script',
  'mlflow.source.git.branch': 'jotaylo/split_train_script',
  'azureml.git.commit': '3df51833d143b722e578d0ae84181cb63bf78747',
  'mlflow.source.git.commit': '3df51833d143b722e578d0ae84181cb63bf78747',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReference

In [8]:
metrics = run.get_metrics()
for k, v in metrics.items():
        print(k, v)

alpha 0.3
mse 3302.673633401725


In [9]:
for file in run.get_file_names():
    print(file)

azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/8_azureml.log
outputs/parameters.json
